# EvoFill working demo

ver 4.   new imputation loss with evo loss

ver 4.1  long range modules integrated in stage1 training

ver 4.2  stage 3 fine tuning with under-reprensted population samples.

last update: 2025/11/13

## 0. Dependency

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

import sys
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
import mamba_ssm

print('Python ver:  ', sys.version)
print('Pytorch ver: ', torch.__version__)
print('Mamba ver:   ', mamba_ssm.__version__)
print('GPU in use:  ', torch.cuda.get_device_name(torch.cuda.current_device()))

os.chdir('/mnt/qmtang/EvoFill/')
from src.data import GenotypeEncoder, ImputationDataset
from src.model import EvoFill
from src.utils import  precompute_maf, metrics_by_maf, print_maf_stat_df, metrics_by_maf_with95ci, print_maf_stat_df_with95ci

Python ver:   3.10.19 | packaged by conda-forge | (main, Oct 13 2025, 14:08:27) [GCC 14.3.0]
Pytorch ver:  2.8.0+cu129
Mamba ver:    2.2.5
GPU in use:   NVIDIA H100 PCIe


In [ ]:
work_dir = Path('/mnt/qmtang/EvoFill_data/20251206_HLA/')
os.chdir(work_dir)

## 1. Encoding all vcfs

### 1.1 1kGP cohorts

In [ ]:
gt_enc = GenotypeEncoder(phased = False, gts012 = False, save2disk = True, save_dir = Path(work_dir / "train"))
gt_enc = gt_enc.encode_new(vcf_path   = "train/HLA_major_pops.vcf.gz" ,
                           evo_mat    = "train/evo_mat_major_pops.tsv")

print(f"[DATA] {gt_enc.n_samples:,} Samples")
print(f"[DATA] {gt_enc.n_variants:,} Variants Sites")
print(f"[DATA] {gt_enc.seq_depth} seq_depth")

[DATA] 总计 190,184 个位点  
[DATA] EvoMat shape: (4526, 4526)
[DATA] 结果已写入 /mnt/qmtang/EvoFill_data/20251205_chr22/train
[DATA] 位点矩阵 = (4526, 190184)，稀疏度 = 22.42%
[DATA] 位点字典 = {'0|0': 0, '0|1': 1, '1|1': 2, '.|.': -1}，字典深度 = 4
[DATA] 4,526 Samples
[DATA] 190,184 Variants Sites
[DATA] 4 seq_depth


### 1.2 aDNA cohorts

In [4]:
gt_enc_aug = GenotypeEncoder(phased=False, gts012=False, save2disk = True, save_dir = Path(work_dir / "augment"))
gt_enc_aug = gt_enc_aug.encode_ref(
        ref_meta_json = work_dir/"train"/"gt_enc_meta.json",   # 与 Stage1 同构
        vcf_path      = "augment/HLA_aDNA_pops.vcf.gz",
        evo_mat       = "augment/evo_mat_aDNA.tsv")

print(f"[DATA] {gt_enc_aug.n_samples:,} Samples")
print(f"[DATA] {gt_enc_aug.n_variants:,} Variants Sites")
print(f"[DATA] {gt_enc_aug.seq_depth} seq_depth")

[DATA] 总计 76,844 个位点  
[DATA] EvoMat shape: (1406, 1406)
[DATA] 结果已写入 /mnt/qmtang/EvoFill_data/20251205_HLA/augment
[DATA] 位点矩阵 = (1406, 76844)，稀疏度 = 18.78%，缺失率 = 0.00%
[DATA] 位点字典 = {'0|0': 0, '0|1': 1, '1|1': 2, '.|.': -1}，字典深度 = 4
[DATA] 1,406 Samples
[DATA] 76,844 Variants Sites
[DATA] 4 seq_depth


### 1.3 finetuning

In [6]:
gt_enc_urp = GenotypeEncoder(phased=False, gts012=False, save2disk=True, save_dir = Path(work_dir / "finetune"))
gt_enc_urp = gt_enc_urp.encode_ref(
        ref_meta_json = work_dir/"train"/"gt_enc_meta.json",   # 与 Stage1 同构
        vcf_path      = work_dir/"finetune"/"HLA_minor_pops.10pct.vcf.gz",
        evo_mat       = work_dir/"finetune"/"evo_mat_minor_pops.10pct.tsv")

print(f'[URP] {gt_enc_urp.n_samples} samples, {gt_enc_urp.n_variants} variants')

[DATA] 总计 76,844 个位点  
[DATA] EvoMat shape: (29, 29)
[DATA] 结果已写入 /mnt/qmtang/EvoFill_data/20251205_HLA/finetune
[DATA] 位点矩阵 = (29, 76844)，稀疏度 = 32.31%，缺失率 = 0.00%
[DATA] 位点字典 = {'0|0': 0, '0|1': 1, '1|1': 2, '.|.': -1}，字典深度 = 4
[URP] 29 samples, 76844 variants


### 1.4 validation

In [3]:
gt_enc_imp = GenotypeEncoder(phased=False, gts012=False, save2disk=True, save_dir = Path(work_dir / "impute_in"))
gt_enc_imp = gt_enc_imp.encode_ref(
        ref_meta_json = work_dir/"train"/"gt_enc_meta.json",   # 与 Stage1 同构
        vcf_path      = work_dir/"impute_in"/"minor_pop_CDX.90pct.masked90p.vcf.gz" )

print(f'[INFER] {gt_enc_imp.n_samples} samples, {gt_enc_imp.n_variants} variants')

[DATA] 总计 76,844 个位点  
[DATA] 结果已写入 /mnt/qmtang/EvoFill_data/20251205_HLA/impute_in
[DATA] 位点矩阵 = (85, 76844)，稀疏度 = 92.99%，缺失率 = 90.02%
[DATA] 位点字典 = {'0|0': 0, '0|1': 1, '1|1': 2, '.|.': -1}，字典深度 = 4
[INFER] 85 samples, 76844 variants


## 2. Training with multi-GPUs

### 2.1 Pre-training

In [ ]:
%%bash
cd /mnt/qmtang/EvoFill/
nohup env OMP_NUM_THREADS=4 \
  accelerate launch --config_file ds_zero3.yaml \
  stage1_training_ds.py \
  > logs/pre_HLA_251206.log 2>&1 &
%%!

### 2.1b

In [ ]:
%%bash
cd /mnt/qmtang/EvoFill/
nohup env OMP_NUM_THREADS=4 CUDA_VISIBLE_DEVICES=1 \
  accelerate launch --config_file ds_zero3.yaml \
  stage2_augmentation_ds.py \
  > logs/post_HLA_251205.log 2>&1 &

### 2.2 Fine-tuning (Few-shot URP)

In [ ]:
%%bash
cd /mnt/qmtang/EvoFill/
nohup env OMP_NUM_THREADS=8 CUDA_VISIBLE_DEVICES=1 \
  accelerate launch --config_file ds_zero3.yaml \
  stage3_finetuning_ds.py \
  > logs/finetune_HLA_251205.log 2>&1 &

### 2.3 Merge weight files

In [ ]:
%%bash
cd /mnt/qmtang/EvoFill_data/20251204_chr22/models/checkpoint-stage1/
python zero_to_fp32.py . ./

## 3. Imputation

### 3.1 Load the trained model

Choose a path where including `<work_dir>/model` and have trained model.

In [ ]:
work_dir = Path('/mnt/qmtang/EvoFill_data/20251206_HLA')
os.chdir(work_dir)
print(f"Work Dir: {work_dir}")

# ---- 1. 加载模型 ----
device = 'cuda' if torch.cuda.is_available() else 'cpu'

json_path = f"{work_dir}/models/model_meta.json"
meta = json.load(open(json_path))
model = EvoFill(
    n_alleles=int(meta["alleles"]),
    total_sites=int(meta["total_sites"]),
    chunk_size=int(meta["chunk_size"]),
    chunk_overlap=int(meta["overlap"]),
    d_model=int(meta["d_model"]),
    d_state=int(meta["d_state"]),
    headdim=int(meta["headdim"]),
).to(device)


# state_dict = torch.load(f"{work_dir}/models/pytorch_model_stage1.bin", map_location="cpu")

state_dict = torch.load("/mnt/qmtang/EvoFill_data/20251206_HLA/models/pytorch_model_stage1.bin", map_location="cpu")

model.load_state_dict(state_dict)
total_params = sum(p.numel() for p in model.parameters())
model.eval()
print(f'[INF] Model[{meta["model_name"]}] loaded.')
print(f"[INF] Total params: {total_params:,}")

Work Dir: /mnt/qmtang/EvoFill_data/20251205_HLA
[INF] Model[hg38_HLA] loaded.
[INF] Total params: 6,650,471


### 3.2 Inferring

In [13]:
gt_enc_imp = GenotypeEncoder.loadfromdisk(Path(work_dir / "impute_in"))

print(f'[INFER] {gt_enc_imp.n_samples} samples, {gt_enc_imp.n_variants} variants')

# ---- 2. 构建推理 Dataset / Loader ----
imp_dataset = ImputationDataset(
    x_gts_sparse=gt_enc_imp.X_gt,
    seq_depth=gt_enc_imp.seq_depth,
    indices=None                 # 可传入指定样本索引
)
imp_dataset.print_missing_stat()          # 查看原始缺失比例

def collate_fn(batch):
    x_onehot = torch.stack([item[0] for item in batch])
    real_idx_list = [item[1] for item in batch]
    return x_onehot, real_idx_list   # 无 y

imp_loader = torch.utils.data.DataLoader(
    imp_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=collate_fn
)

y_prob = []
y_mask = []
with torch.no_grad():
    for x_onehot, real_idx in tqdm(imp_loader, desc='Imputing'):
        x_onehot = x_onehot.to(device)
        _, prob, _ = model(x_onehot)
        miss_mask = x_onehot[..., -1].bool()
        y_prob.append(prob)
        y_mask.append(miss_mask)
y_prob = torch.cat(y_prob, dim=0).cpu().numpy()
y_mask = torch.cat(y_mask, dim=0).cpu().numpy()
# 4. 保存
out_dir = os.path.join(work_dir, 'impute_out')
os.makedirs(out_dir, exist_ok=True)
np.save(os.path.join(out_dir, 'impute_prob.npy'), y_prob)
np.save(os.path.join(out_dir, 'impute_mask.npy'), y_mask)
print(f'[INF] 概率矩阵已保存 → {out_dir}/impute_prob.npy '
      f'with shape = {y_prob.shape} ')


[INFER] 85 samples, 76844 variants
[ImputationDataset] 85 samples, missing rate = 90.02%


Imputing: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


[INF] 概率矩阵已保存 → /mnt/qmtang/EvoFill_data/20251205_HLA/impute_out/impute_prob.npy with shape = (85, 76844, 3) 


### 3.4 Evaluating the imputation results

In [14]:
gt_enc_true = GenotypeEncoder(phased=False, gts012=False, save2disk=False)
gt_enc_true = gt_enc_true.encode_ref(
        ref_meta_json = work_dir/"train"/"gt_enc_meta.json",   # 与 Stage1 同构
        vcf_path      = work_dir/"impute_out"/"minor_pop_CDX.90pct.vcf.gz" )
y_true = gt_enc_true.X_gt.toarray()
maf, bin_cnt = precompute_maf(y_true)
y_true_oh = np.eye(gt_enc_true.seq_depth - 1)[y_true]
bins_metrics   = metrics_by_maf(y_prob, y_true_oh, hap_map = gt_enc_true.hap_map, maf_vec = maf, mask=y_mask)
print_maf_stat_df(bin_cnt,{'val': bins_metrics})

[DATA] 总计 76,844 个位点  
[DATA] 位点矩阵 = (85, 76844)，稀疏度 = 29.78%，缺失率 = 0.00%
[DATA] 位点字典 = {'0|0': 0, '0|1': 1, '1|1': 2, '.|.': -1}，字典深度 = 4
     MAF_bin Counts val_Acc val_INFO val_IQS val_MaCH
(0.00, 0.05)  34138   0.988    0.171   0.154    0.860
(0.05, 0.10)  10823   0.950    0.697   0.724    0.831
(0.10, 0.20)   9929   0.911    0.713   0.721    0.838
(0.20, 0.30)   8136   0.918    0.814   0.834    0.941
(0.30, 0.40)   6965   0.907    0.828   0.841    0.965
(0.40, 0.50)   6601   0.918    0.850   0.869    0.987


In [15]:
bins_metrics   = metrics_by_maf_with95ci(y_prob, y_true_oh, hap_map = gt_enc_true.hap_map, maf_vec = maf, mask=y_mask)
print_maf_stat_df_with95ci(bin_cnt,{'val': bins_metrics})

     MAF_bin Counts val_Acc val_INFO val_IQS val_MaCH   val_Acc_CI95  val_INFO_CI95   val_IQS_CI95  val_MaCH_CI95
(0.00, 0.05)  34138   0.988    0.171   0.154    0.860 [0.988, 0.988] [0.168, 0.174] [0.151, 0.158] [0.857, 0.863]
(0.05, 0.10)  10823   0.950    0.697   0.724    0.831 [0.949, 0.951] [0.691, 0.703] [0.718, 0.730] [0.825, 0.837]
(0.10, 0.20)   9929   0.911    0.713   0.721    0.838 [0.910, 0.913] [0.708, 0.719] [0.714, 0.727] [0.832, 0.844]
(0.20, 0.30)   8136   0.918    0.814   0.834    0.941 [0.916, 0.921] [0.809, 0.819] [0.828, 0.839] [0.937, 0.945]
(0.30, 0.40)   6965   0.907    0.828   0.841    0.965 [0.904, 0.910] [0.823, 0.832] [0.836, 0.846] [0.962, 0.968]
(0.40, 0.50)   6601   0.918    0.850   0.869    0.987 [0.915, 0.921] [0.846, 0.854] [0.865, 0.873] [0.985, 0.989]


chr22 AADR 中总计包含 14,867 个位点（删去1kGP中不存在、非 biallelic 位点）

测试集：90% 少数族裔样本（CDX, BEB, ASW），90% masked 位点

训练集：

**= 1kGP ONLY =**

|      MAF_bin | Counts | val_Acc | val_INFO | val_IQS | val_MaCH |
| :----------: | :----: | ------: | -------: | ------: | -------: |
| (0.00, 0.05) |   1546 |   0.972 |    0.060 |   0.068 |    0.166 |
| (0.05, 0.10) |   1246 |   0.931 |    0.198 |   0.216 |    0.367 |
| (0.10, 0.20) |   2205 |   0.875 |    0.332 |   0.353 |    0.545 |
| (0.20, 0.30) |   3112 |   0.798 |    0.487 |   0.495 |    0.754 |
| (0.30, 0.40) |   5193 |   0.755 |    0.550 |   0.552 |    0.853 |
| (0.40, 0.50) |   1565 |   0.739 |    0.548 |   0.545 |    0.845 |

**= aDNA -> 1kGP =** 

|      MAF_bin | Counts | val_Acc | val_INFO | val_IQS | val_MaCH |
| :----------: | :----: | ------: | -------: | ------: | -------: |
| (0.00, 0.05) |   1546 |   0.972 |    0.089 |   0.085 |    0.193 |
| (0.05, 0.10) |   1246 |   0.935 |    0.280 |   0.283 |    0.463 |
| (0.10, 0.20) |   2205 |   0.890 |    0.452 |   0.456 |    0.685 |
| (0.20, 0.30) |   3112 |   0.824 |    0.592 |   0.574 |    0.850 |
| (0.30, 0.40) |   5193 |   0.787 |    0.645 |   0.614 |    0.906 |
| (0.40, 0.50) |   1565 |   0.770 |    0.634 |   0.599 |    0.894 |

**= 1kGP -> aDNA =** 

|      MAF_bin | Counts | val_Acc | val_INFO | val_IQS | val_MaCH |
| :----------: | :----: | ------: | -------: | ------: | -------: |
| (0.00, 0.05) |   1546 |   0.938 |    0.109 |   0.227 |    0.734 |
| (0.05, 0.10) |   1246 |   0.893 |    0.174 |   0.396 |    0.633 |
| (0.10, 0.20) |   2205 |   0.810 |    0.196 |   0.435 |    0.545 |
| (0.20, 0.30) |   3112 |   0.690 |    0.144 |   0.433 |    0.499 |
| (0.30, 0.40) |   5193 |   0.640 |    0.101 |   0.429 |    0.503 |
| (0.40, 0.50) |   1565 |   0.604 |    0.102 |   0.398 |    0.464 |


### 3.5 Saving to .vcf

In [38]:
from cyvcf2 import VCF, Writer

# 0. 路径
ref_vcf = "/mnt/qmtang/EvoFill_data/20251107_ver4/minor_pops.masked30p.vcf.gz"
out_vcf = os.path.join(out_dir, 'imputed.vcf.gz')

n_site = gt_enc.n_variants
n_samp = gt_enc.n_samples
n_alleles = gt_enc.seq_depth - 1
assert y_prob.shape == (n_samp, n_site, n_alleles)

# 2. 反向映射  idx -> '0|0' / '0|1' / ...
rev_hap_map = {v: k for k, v in gt_enc.hap_map.items()}

samp2idx = {sid: i for i, sid in enumerate(gt_enc.sample_ids)}

# 4. 打开参考 VCF
invcf = VCF(ref_vcf)
tmpl  = invcf
tmpl.set_samples(gt_enc.sample_ids)   # 替换样本列

out = Writer(out_vcf, tmpl, mode='wz')

for rec_idx, rec in enumerate(invcf):
    # 当前位点全部样本的 GT
    gt_int_pairs = []
    for samp_idx, sample_id in enumerate(gt_enc.sample_ids):
        old_gt = rec.genotypes[samp_idx]          # [allele1, allele2, phased]
        if old_gt[0] == -1 or old_gt[1] == -1:    # 缺失
            prob_vec = y_prob[samp_idx, rec_idx, :].ravel()
            best_idx = int(prob_vec.argmax())
            gt_str   = rev_hap_map[best_idx]
            alleles  = list(map(int, gt_str.split('|')))
            phased   = old_gt[2] if old_gt[2] != -1 else 1
            gt_int_pairs.append([alleles[0], alleles[1], phased])
        else:                                       # 非缺失，保持原样
            gt_int_pairs.append(old_gt)

    # 转成 int8 二维数组  (n_sample, 3)  last dim = [a1,a2,phased]
    gt_array = np.array(gt_int_pairs, dtype=np.int8)
    rec.set_format('GT', gt_array)
    out.write_record(rec)

invcf.close()
out.close()

# 5. tabix
os.system(f'tabix -fp vcf {out_vcf}')
print(f'[INF] 缺失位点填充完成 → {out_vcf}')

[INF] 缺失位点填充完成 → /mnt/qmtang/EvoFill_data/20251107_ver4/impute_out/imputed.vcf.gz
